In [6]:
import os
import numpy as np
import pandas as pd
import random, math
import subprocess

In [14]:
np.iinfo(np.uint64).max

18446744073709551615

In [7]:
O = np.matrix([[0,0,1,1,0,0,1,0]
              ,[1,0,1,0,0,1,0,1]
              ,[1,0,1,0,0,1,0,0]
              ,[0,0,1,1,0,0,1,0]
              ,[0,0,1,0,0,0,1,0]
              ,[0,0,1,1,0,0,1,0]
              ,[0,0,1,1,0,0,0,0]
              ,[1,0,1,0,0,1,0,1]
              ,[0,0,1,0,0,0,1,0]
              ,[0,0,0,0,0,0,0,0]])

In [8]:
n,m = O.shape
cell_names = ['cell'+str(i) for i in range(n)]
mut_names = ['mut'+str(i) for i in range(m)]
df = pd.DataFrame(O)
df.columns = mut_names
df.index = cell_names
df.index.name = 'cellID/mutID'
df.to_csv('O.txt', sep='\t')

In [10]:
%%time
%%bash

/data/frashidi/_Archived/1_PhISCS/csp-test/csp_maxsat \
    -s /data/frashidi/PhISCS/PhISCS/PhISCS-B/solvers/maxino/maxino-2015-k16-static -i \
    -f O.txt \
    -n 0.20 \
    -p 0.00010 \
    -o .

par_fnWeight	12206.1
par_fnWeight_neg	13592.4
par_fpWeight	4605.17
par_fpWeight_neg	13815.4
CPU times: user 1.91 ms, sys: 5.76 ms, total: 7.67 ms
Wall time: 25.2 ms


In [ ]:
%%bash

python /data/frashidi/PhISCS/code/PhISCS_ILP/PhISCS_ILP.py \
    -SCFile test.txt -fn 0.020 -fp 0.00010 \
    -o . -w 0 -kmax 0

–––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

In [ ]:
def important_columns(D):
    ic = defaultdict(lambda: 0)
    for p in range(D.shape[1]):
        for q in range(p + 1, D.shape[1]):
            conf_oneone = []
            conf_zeroone = []
            conf_onezero = []
            for r in range(D.shape[0]):
                if D[r][p] == 1 and D[r][q] == 1:
                    conf_oneone.append(r)
                if D[r][p] == 0 and D[r][q] == 1:
                    conf_zeroone.append(r)
                if D[r][p] == 1 and D[r][q] == 0:
                    conf_onezero.append(r)
            for r1 in conf_oneone:
                for r2 in conf_zeroone:
                    for r3 in conf_onezero:
                        print(p,q, r1, r2, r3)
                        ic[p] += 1
                        ic[q] += 1
    return sorted(ic.items(), key=operator.itemgetter(1), reverse=True)

print(important_columns(noisy))

In [2]:
# import pandas as pd

# df = pd.DataFrame(noisy)
# df.columns = ['mut'+str(i) for i in range(noisy.shape[1])]
# df.index = ['cell'+str(i) for i in range(noisy.shape[0])]
# df.index.name = 'cellID/mutID'
# df.to_csv('noisy.SC', sep='\t')

# df = pd.read_csv(file, index_col=0, sep='\t')

[1] [ms package (paper)](https://academic.oup.com/bioinformatics/article/18/2/337/225783)  
[2] [ms package (download)](http://home.uchicago.edu/~rhudson1/source/mksamples.html)  
[3] [csp solvers](http://mse17.cs.helsinki.fi/descriptions.html)  
[4] [PhISCS](https://www.biorxiv.org/content/early/2018/07/25/376996)  

For installing maxino go to the url of [3] and then `wget` the `maxino` package. Then `unzip` it. Ater that go to the code folder and just run `make`. Then change `csp_solver_path` accordingly.  

n = number of cells  
m = number of mutations  
seed = is a seed number of generating the ground truth by ms package (not important leave it as 1)  
fn = false negative rate  
fp = false positive rate  
na = missing value rate